# Module 3: Bedrock Inline Agents with MCP Integration (45 minutes)

## Overview
In this module, we will rebuild the Film Agent using Bedrock Inline Agents, a lightweight framework that allows users to dynamically configure an agent at runtime, enabling rapid prototyping and debugging. To learning more visit [Configure an inline agent at runtime](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create-inline.html).Then we will explore building and integrating Model Context Protocol (MCP) with Bedrock Inline agents.

---

Amazon Bedrock Inline Agents provide a flexible way to create AI agents that can be configured at runtime. This approach offers several advantages:

- **Dynamic Configuration**: Agents can be configured on-the-fly without redeployment
- **Rapid Prototyping**: Quick iteration on agent capabilities and behavior
- **Debugging**: Better visibility into agent execution and decision-making
- **Tool Integration**: Easy integration with custom tools and external services

MCP is a standardized integration framework designed for AI agents, enabling them to seamlessly connect and communicate with external tools, data sources, and services in real time. It acts as a universal connector-much like how USB-C standardizes device connectivity-MCP defines clear roles and message formats so that agents (as MCP Hosts) can discover, request, and interact with resources provided by MCP Servers, with MCP Clients facilitating the communication between them.

**WARNING: you need to complete module 2 first because some resources like knowledge base are created in module**

In [ ]:
from InlineAgent.knowledge_base import KnowledgeBasePlugin
from InlineAgent.agent import InlineAgent
from InlineAgent.action_group import ActionGroup
import boto3
from boto3.dynamodb.conditions import Key, Attr
import nest_asyncio
import time
import json

## DynamoDB parameters
dynamodb_resource = boto3.resource('dynamodb')
## Rekognition parameters
rek_client = boto3.client('rekognition')

## Creating Knowledge Base

Knowledge bases are a key component of Bedrock Agents, allowing them to access and retrieve information from external data sources. In this section, we'll:

1. Load the knowledge base configuration from previous modules
2. Create a KnowledgeBasePlugin instance that connects to our existing knowledge base
3. Configure vector search parameters to optimize retrieval performance

The knowledge base contains information about films, directors, actors, and other movie-related data that our agent can use to answer user queries.

In [ ]:
%store -r knowledge_base_name
%store -r kb_config
%store -r agent_instruction
%store -r agent_name

In [ ]:
film_kb = KnowledgeBasePlugin(
    name=knowledge_base_name,
    description=kb_config["kb_instruction"],
    additional_props={
        "retrievalConfiguration": {"vectorSearchConfiguration": {"numberOfResults": 1}}
    },
)

In [ ]:
# Step 3: Define agent
agent = InlineAgent(
    foundation_model="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    instruction=agent_instruction,
    knowledge_bases=[film_kb],
    agent_name=f"inline-{agent_name}",
    user_input=True,
)

In [ ]:
# Step 4: Invoke agent
result = await agent.invoke(input_text="which film is directored by Curtis Clark",
                            enable_trace=False
                           )

## Creating Action Group with Direct Function Integration

Action Groups are logical groupings of tools that an agent can use to perform specific tasks. In this section, we'll create an action group that enables our agent to detect celebrities in videos using AWS Rekognition.

The implementation follows these steps:

1. Load configuration parameters for DynamoDB and S3 video path
2. Implement helper functions for celebrity detection and DynamoDB queries
3. Create a main `detect_key_figures` function that:
   - Processes videos stored in S3
   - Uses AWS Rekognition to identify celebrities
   - Enriches detection results with cast information from DynamoDB
4. Package the function into an ActionGroup that can be used by our agent

This approach directly integrates the function with the agent, allowing for seamless tool invocation.

In [ ]:
%store -r cast_table
%store -r cast_pk
%store -r film_video_s3_path

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import time
import json

def get_cast_member(cast_id):
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception as e:
        print(f'Error querying table: {cast_table}.')
        print(str(e))

def start_celebrity_detection(bucket, video_key):
    response = rek_client.start_celebrity_recognition(
        Video={
            'S3Object': {
                'Bucket': bucket,
                'Name': video_key
            }
        }
    )
    return response['JobId']

def get_celebrity_detection_results(job_id):
    response = rek_client.get_celebrity_recognition(JobId=job_id)
    return response

def extract_bucket_key(video_s3_path):
    path = video_s3_path[5:]  # Remove 's3://'
    bucket, key = path.split('/', 1)  # Split into bucket and key   
    return bucket, key

def detect_key_figures(video_s3_path:str) -> list:
    """
    Detect key figures and their cast role in a video

    Parameters:
        video_s3_path: s3 path to the video, e.g., s3://bucket/prefix/video_file

    Returns:
        list: a list of dictionaries with attributes about the key figure
    """
    
    bucket, key = extract_bucket_key(video_s3_path)
    
    job_id = start_celebrity_detection(bucket, key)
    print(f"Started celebrity detection job: {job_id}")

    while True:
        response = get_celebrity_detection_results(job_id)
        status = response['JobStatus']
        
        if status in ['SUCCEEDED', 'FAILED']:
            print("JOB COMPLETE....")
            break
        
        print("Job in progress...")
        time.sleep(10)

    unique_celebrities = {}  # Dictionary to store unique celebrities

    if status == 'SUCCEEDED':
        for celebrity in response['Celebrities']:
            celeb = celebrity['Celebrity']
            
            # Only process celebrities with 95%+ confidence
            if celeb['Confidence'] >= 95.0:
                celeb_id = celeb['Id']
                
                # Store or update celebrity info only if not already stored
                if celeb_id not in unique_celebrities:
                    celebrity_info = {
                        'name': celeb['Name'],
                        'confidence': celeb['Confidence'],
                        'id': celeb_id,
                        'first_appearance': celebrity['Timestamp']
                    }
                    # If you have additional celebrity info in DynamoDB
                    try:
                        query_items = get_cast_member(celeb_id)
                        if query_items:
                            celebrity_info.update(query_items[0])
                    except Exception as e:
                        print(f"Error fetching additional celebrity info: {str(e)}")
                    
                    unique_celebrities[celeb_id] = celebrity_info
    else:
        print("Detection failed....")
        return ""
    # Convert the dictionary values to a list for the final output
    return json.dumps(list(unique_celebrities.values()))

In [ ]:
# Step 2: Logically group tools together
key_figures_action_group = ActionGroup(
    name="KeyFiguresActionGroup",
    description="This is action group to identify key figures and look up cast members and their role from the video",
    tools=[detect_key_figures],
)

In [ ]:
# Step 3: Define agent
agent = InlineAgent(
    foundation_model="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    instruction=agent_instruction,
    knowledge_bases=[film_kb],
    agent_name=f"inline-{agent_name}",
    action_groups=[key_figures_action_group],
    user_input=True,
)

In [ ]:
%%time
# Step 4: Invoke agent
result = await agent.invoke(input_text=f"Who are the celebrities in this video: {film_video_s3_path}",
                            enable_trace=False
                           )

## Creating MCP Action Group

Model Context Protocol (MCP) provides a standardized way for foundation models to interact with external tools and services. In this section, we'll implement the same celebrity detection functionality using MCP, which offers several advantages:

1. **Decoupling**: The tool implementation is completely separate from the agent code
2. **Scalability**: MCP servers can be deployed independently and scaled as needed
3. **Reusability**: The same MCP tools can be used by multiple agents
4. **Flexibility**: Tools can be implemented in any language that supports the MCP protocol

We'll follow these steps:
1. Create an MCP server that exposes the celebrity detection functionality
2. Connect our agent to the MCP server using the MCP client
3. Create an action group that uses the MCP tools
4. Invoke the agent with the MCP-based action group

This approach demonstrates how to extend agent capabilities through external services.

### MCP Server Implementation

Below is the implementation of our MCP server (`key_figures_mcp_server.py`). This script:

1. Creates a FastMCP server that exposes the `detect_key_figures` tool
2. Initializes AWS clients for Rekognition and DynamoDB
3. Implements helper functions for celebrity detection and database queries
4. Defines the main tool function with proper input/output schema
5. Handles asynchronous processing using `asyncio.to_thread`

The MCP server can be run as a standalone process and communicates with the agent through the MCP protocol. This separation of concerns allows for more modular and maintainable code.

In [ ]:
%pycat key_figures_mcp_server.py

### Setting Up MCP Client

To connect our agent to the MCP server, we need to:

1. Set up the MCP client configuration
2. Pass AWS credentials to the MCP server
3. Configure the communication channel between the agent and the server

We'll use the `StdioServerParameters` class to launch the MCP server as a subprocess and communicate with it through standard input/output. This approach is ideal for development and testing scenarios.

In [ ]:
from mcp import StdioServerParameters
import asyncio

from InlineAgent.tools import MCPStdio

# Get SageMaker Execution Role Credential
def get_credentials():
    session = boto3.Session()
    credentials = session.get_credentials()
    frozen_creds = credentials.get_frozen_credentials()
    region = session.region_name
    
    return {
        "AWS_ACCESS_KEY_ID": frozen_creds.access_key,
        "AWS_SECRET_ACCESS_KEY": frozen_creds.secret_key,
        "AWS_SESSION_TOKEN": frozen_creds.token,
        "AWS_DEFAULT_REGION": region
    }

### Running the Agent with MCP Integration

Now we'll execute our agent with the MCP-based action group. The process involves:
1. Starting the MCP server as a subprocess
2. Creating an MCP client that connects to the server
3. Creating an action group that uses the MCP client
4. Configuring the agent with the MCP action group
5. Invoking the agent with a query about celebrities in the video
6. Properly cleaning up resources when done

In [ ]:
nest_asyncio.apply()

# Step 1: Define MCP stdio parameters
server_params = StdioServerParameters(
    command="python",
    args=[
        "key_figures_mcp_server.py",
    ],
    env={"CAST_TABLE": cast_table, "CAST_PK":cast_pk, **get_credentials()},
)

async def run_agent():
    
    key_figure_mcp_client = await MCPStdio.create(server_params=server_params)

    try:
        key_figure_mcp_action_group = ActionGroup(
            name="KeyFiguresMCPActGroup",
            description="This is action group to identify key figures and look up cast members and their role from the video",
            mcp_clients=[key_figure_mcp_client],
        )

        await InlineAgent(
            foundation_model="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            instruction=agent_instruction,
            knowledge_bases=[film_kb],
            agent_name=f"inline-{agent_name}",
            action_groups=[key_figure_mcp_action_group],
            user_input=True,
        ).invoke(
            input_text=f"Who are the celebrities in this video: {film_video_s3_path}"
        )

    finally:
        await key_figure_mcp_client.cleanup()

# For Jupyter notebook execution
await run_agent()

## Conclusion

In this notebook, we've demonstrated two approaches for integrating tools with Amazon Bedrock Inline Agents:

1. **Direct Function Integration**: We implemented celebrity detection directly in the notebook and packaged it as an action group. This approach is simpler and works well for lightweight tools.
2. **MCP Integration**: We implemented the same functionality as an MCP server and connected it to our agent. This approach provides better separation of concerns and allows for more complex tool implementations.

Both approaches successfully detected celebrities in the video and enriched the results with cast information from DynamoDB. The agent was able to use these tools to provide meaningful responses to user queries.

Key takeaways:

- Bedrock Inline Agents provide a flexible framework for building AI agents
- Action groups allow for logical grouping of related tools
- MCP enables clean separation between agent logic and tool implementation
- Knowledge bases enhance agent capabilities with external information
- Observability features help track agent performance and behavior

These patterns can be extended to build more complex agents with diverse capabilities across various domains.